In [29]:
import os
import numpy as np
import pandas as pd
#feature extraction in pipeline--------->
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [30]:
ppt=[]
os.chdir("S:\\op_spam_train\\positive_polarity\\true")
for filename in os.listdir(os.getcwd()):
    with open(filename,'r') as file:
        ppt.append(file.read())
ppf=[]
os.chdir("S:\\op_spam_train\\positive_polarity\\false")
for filename in os.listdir(os.getcwd()):
    with open(filename,'r') as file:
        ppf.append(file.read())
        
npt=[]
os.chdir("S:\\op_spam_train\\negative_polarity\\true")
for filename in os.listdir(os.getcwd()):
    with open(filename,'r') as file:
        npt.append(file.read())

npf=[]
os.chdir("S:\\op_spam_train\\negative_polarity\\false")
for filename in os.listdir(os.getcwd()):
    with open(filename,'r') as file:
        npf.append(file.read())

deceptive=[]
deceptive=ppf+npf
truthful=[]
truthful=ppt+npt
data=[]
data=deceptive+truthful
data=np.array(data)
#train_data contains all the deceptive reviews from the positive part as a list
#train_data[0]='first review' and so on

In [31]:
features=['review']

data_df = pd.DataFrame(data, columns=features)

for label, row in data_df.iterrows():
    data_df.loc[label, 'review_len']=len(row['review'])

index=np.arange(1600)
data_df['index'] = index

for label, row in data_df.iterrows():
    if (int(row['index'])<800):
        data_df.loc[label, 'label']=0
    else:
        data_df.loc[label, 'label']=1


data_df

,review,review_len,index,label
0,"After recent week stay at the Affinia Hotels, ...",362.0,0,0.0
1,"Although much too overpriced in my opinion, th...",302.0,1,0.0
2,The Affinia hotel in Chicago was superb. the r...,680.0,2,0.0
3,THIS HOTEL IS FANTASTIC. I stayed there on my ...,423.0,3,0.0
4,The Affinia Chicago is a wonderful place to st...,304.0,4,0.0
5,The magnificent mile in Chicago is a great pla...,416.0,5,0.0
6,My stay at the Affinia last weekend was ideal ...,431.0,6,0.0
7,We recently completed our first visit to Chica...,451.0,7,0.0
8,We stayed for two nights for a meeting. It is ...,317.0,8,0.0
9,"I just came back from this awesome, terrific h...",1100.0,9,0.0


In [32]:
X = data_df.review
y = data_df.label
print(X.shape)
#right now X is a 1-d we will pass it to the vectorizer to convert it into 2-d
print(y.shape)

(1600,)
(1600,)


In [33]:
# Now we split X and y into training and testing sets

from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1120,)
(480,)
(1120,)
(480,)


In [34]:
#Vectorizing the dataset

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,2))

X_train_dtm = vect.fit_transform(X_train)

X_test_dtm = vect.transform(X_test)


mnb = MultinomialNB()
mnb.fit(X_train_dtm, y_train)

#making pedictions
y_predict = mnb.predict(X_test_dtm)

from sklearn import metrics 
print(metrics.accuracy_score(y_test, y_predict))
print(metrics.confusion_matrix(y_test, y_predict))
'''Confusion matrix 
[TN FP 
FN TP]'''


0.9
[[234  17]
 [ 31 198]]


'Confusion matrix \n[TN FP \nFN TP]'

In [38]:
'''Pipepline implementation'''
pipeline = Pipeline([
        ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
        ('counts_ngram', CountVectorizer(ngram_range=(1,5), analyzer='char')),
        ('tf_idf_ngram', TfidfTransformer())
    ])),
  ])),
  ('classifier', MultinomialNB())
])

pipeline.fit(X_train, y_train)
pipeline.predict(X_test)
print(metrics)
print(metrics.confusion_matrix(y_test, y_predict))

<module 'sklearn.metrics' from 'C:\\Users\\.hp\\Anaconda3\\lib\\site-packages\\sklearn\\metrics\\__init__.py'>
[[234  17]
 [ 31 198]]
